# Explore Sampling from the Ornstein-Uhlenbeck Process (OU)
The OU process,  in general, takes the form
$$
dX_t = -\alpha(t) (X_t - m(t)) + \sigma(t) dB_t, \quad X_0 = x_0
$$
where $\alpha(t)>0$ and $\sigma(t)>0$, and $B_t$ is a standard Brownian motion.  This is a mean reverting process, as the first term encourages it to return to $m(t)$.

This can be approximated using the Euler-Maruyama time step scheme, which corresponds to
$$
X_{n+1} = X_n - \alpha(t_n) (X_n -m(t_n))\Delta t + \sigma(t_n)\sqrt{\Delta t} \xi_{n+1}, \quad \xi_{n+1}\sim N(0,1)
$$

In [ ]:
using Plots; pyplot()
using Random
using LaTeXStrings
using Statistics
using Printf

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2,ms=8)

In [ ]:
function sample_ou(x0, α, σ, m, Δt, nΔt)
    x_vals = zeros(nΔt + 1);
    x = x0;
    x_vals[1] = x0;
    
    for j in 0:nΔt-1
        t = j * Δt;
        ξ = randn();
        
        x+= -α(t) * (x-m(t)) * Δt + σ(t) * sqrt(Δt) * randn();
        x_vals[j+2] = x;
    end
        
    return  x_vals
    
end

In [ ]:
tmax = 10;
Δt = 0.01;
nΔt = Int(tmax/Δt);

Random.seed!(100);
t_vals = Δt * (0:nΔt);

α= t-> 1;
m =t-> 0;
σ = t-> 1;

x0 = 0.;

x_vals = sample_ou(x0, α, σ, m, Δt, nΔt);

In [ ]:
plot(t_vals, x_vals, label="Sample Path")
xlabel!(L"$t$")

In [ ]:
plot(t_vals, x_vals, label="")
x_vals = sample_ou(x0, α, σ, m, Δt, nΔt);
plot!(t_vals, x_vals, label="")
x_vals = sample_ou(x0, α, σ, m, Δt, nΔt);
plot!(t_vals, x_vals, label="")
xlabel!(L"$t$")

## Examine Path Distribution

In [ ]:
n_samples = 10^2;
tmax = 10;
Δt = 0.01;
nΔt = Int(tmax/Δt);

Random.seed!(100);
t_vals = Δt * (0:nΔt);

α= t-> 1;
m =t-> 0;
σ = t-> 1;

# α= t-> 1;
# m =t-> t;
# σ = t-> 0.01 + 5*sin(π*t)^2;


x0 = 0.;

t_vals = Δt * (0:nΔt);
sample_paths = zeros(n_samples,length(t_vals));


Random.seed!(100);

Threads.@threads for i in 1:n_samples
    sample_paths[i,:] .=sample_ou(x0, α, σ, m, Δt, nΔt);
end


In [ ]:
mean_path= mean(sample_paths, dims=1)[:]; # use [:] to ensure these are column vectors
var_path = var(sample_paths, dims=1)[:];

plot(t_vals, mean_path, ribbon = sqrt.(var_path),label="Mean")
xlabel!(L"$t$")